# Setup

In [ ]:
%pip install kornia==0.4.0
import matplotlib.pyplot as plt
import numpy as np
import cv2
import math 
import torch
from torchvision import datasets, transforms
from torch import nn, optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import kornia as K
import time

     |████████████████████████████████| 204kB 17.2MB/s 
     |████████████████████████████████| 748.8MB 23kB/s 
ERROR: torchvision 0.8.2+cu101 has requirement torch==1.7.1, but you'll have torch 1.6.0 which is incompatible.
  Found existing installation: torch 1.7.1+cu101
    Uninstalling torch-1.7.1+cu101:
      Successfully uninstalled torch-1.7.1+cu101


## Mount drive

In [ ]:
your_path = ''
from google.colab import drive
drive.mount('/content/gdrive')
path = '/content/gdrive/My Drive/'+your_path
import sys
sys.path.append(path)
from utils.points import cords_to_map

Mounted at /content/gdrive


## GPU

In [ ]:
print(torch.__version__)
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
  DEVICE = 'cpu'
  print('CUDA is not available.  Training on CPU ...')
else:
  DEVICE = 'cuda'
  print('CUDA is available!  Training on GPU ...')



1.6.0
CUDA is available!  Training on GPU ...


# DATA

## Define transformatiom

In [ ]:
from datasets.utils.transforms import ColorJitter, ToGray, Rescale,ToTensor, get_twin

## Dataset model

In [ ]:
from datasets.COCO.COCO_model import COCO_dataset



## Generate dataset and dataloader

In [ ]:
transform = transforms.Compose([Rescale((240,320)),
                                ColorJitter(), 
                                ToGray(),
                                ToTensor()]) 

dataset = COCO_dataset(path+'/datasets/COCO/labeled_coco.csv',
                                  path+'/datasets/COCO/val2017/', 
                                  transform=transform, 
                                  landmark_bool=True)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=5, shuffle=True)
'''
print(len(dataset))
print(len(dataloader))
for iter, (im,label) in enumerate(dataloader):
  print('ok')
  if iter>2:
    break
'''

  

"\nprint(len(dataset))\nprint(len(dataloader))\nfor iter, (im,label) in enumerate(dataloader):\n  print('ok')\n  if iter>2:\n    break\n"

## Test model

### plot function

In [ ]:
from utils.plot import plot_imgs


### test

In [ ]:
'''for iter, (im, label) in enumerate(dataloader):
  label = label.type(torch.double)
  label = label.unsqueeze(1)
  imgs= K.tensor_to_image(im)
  size = im.size()
  map = cords_to_map(label, size, device=False)
  im_map = K.tensor_to_image(map)
  if iter%10==0:
    print('iteration {}/{} is running'.format(iter,len(dataloader)))
  plot_imgs(imgs, label=label)
  plot_imgs(im_map)
  if iter>0:
    break
'''

"for iter, (im, label) in enumerate(dataloader):\n  label = label.type(torch.double)\n  label = label.unsqueeze(1)\n  imgs= K.tensor_to_image(im)\n  size = im.size()\n  map = cords_to_map(label, size, device=False)\n  im_map = K.tensor_to_image(map)\n  if iter%10==0:\n    print('iteration {}/{} is running'.format(iter,len(dataloader)))\n  plot_imgs(imgs, label=label)\n  plot_imgs(im_map)\n  if iter>0:\n    break\n"

# Net architecture

## Superpoint model

In [ ]:
from models.superpoint import SuperPointNet

## functions for Loss calculations

In [ ]:
from models.utils import detector_loss, descriptor_loss


# Training Superpoint on COCO

## Train function

In [ ]:
def train_coco_magic(dataloader, net, save_path, filename, lr=0.001):
  t_0 = time.time()
  optimizer = torch.optim.Adam(model.parameters(), lr=lr)
  DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
  net.to(DEVICE)
  model.train()
  N_epoch = 4
  for e in range(N_epoch):
    for iter, (im, label) in enumerate(dataloader):
      optimizer.zero_grad()
      im = im.to(DEVICE).type(torch.float)
      label = label.to(DEVICE)
    
      #get twin im and homography
      twin_im, H = get_twin(im)
      if len(H.size()) < 3:
            H = H.unsqueeze(0)
      H_invert = torch.inverse(H)
      
      #go through model
      chi_points, desc = net(im)
      twin_chi_points, twin_desc = net(twin_im)
      #get map label
      label = label.type(torch.double)
      size = im.size()
      map = cords_to_map(label, size)
      map[map<0.01] = 0
      map[:,:,0:5,0] = 0
      map[:,:,:,0:7] = 0
      map[:,:,-5:,:] = 0
      map[:,:,:,-7:] = 0      

      #get twin map and valid mask
      twin_map = map.type(torch.float)  
      twin_map = K.warp_perspective(twin_map, H, dsize=(im.size(2), im.size(3)))
      valid_mask = torch.ones_like(im, dtype=torch.float32).to(DEVICE)
      valid_mask = K.warp_perspective(valid_mask, H, dsize=(im.size(2), im.size(3)))

        
      #loss
      detector_loss_1 = detector_loss(map, chi_points)
      detector_loss_2 = detector_loss(twin_map, twin_chi_points)
      desc_loss = descriptor_loss(desc, twin_desc, H, H_invert, valid_mask)
      loss = detector_loss_1 + detector_loss_2 + 0.0001*desc_loss
      
      #optimize
      loss.backward()
      optimizer.step()
      
      
      if iter%10==0:
        print('iteration {}/{} is running'.format(e*len(dataloader)+iter,N_epoch*len(dataloader)))
        print('loss is:',loss.item())
      if iter%50==0:
        t_c = time.time()
        minute = (t_c-t_0)/60
        print('saving weights from iteration {} with loss {}, {} minutes pased'.format(e*len(dataloader)+iter,loss.item(),int(minute)))
        print('detector loss 1: {}, detector loss 2: {}, descriptor loss: {}'.format(detector_loss_1,detector_loss_2,desc_loss))
        torch.save(model.state_dict(), save_path+filename)
  # Save weights
  torch.save(model.state_dict(), save_path+filename)
  t_f = time.time()
  hours = (t_f-t_0)/3600
  print('finished in {} hours'.format(hours))

## Run

In [ ]:
#tensorflow
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
import os 

%load_ext tensorboard

logs_base_dir = 'logs'
os.makedirs(logs_base_dir, exist_ok=True)



dataloader = torch.utils.data.DataLoader(dataset, batch_size=3, shuffle=True)
model = SuperPointNet(superpoint_bool=True)
weights_path = path+'/weights/magic_coco_weights.pth'
model.load_state_dict(torch.load(weights_path,
                               map_location=lambda storage, loc: storage))



train_coco_magic(dataloader, model, path, '/weights/super_coco_weights_test.pth')


In [ ]:
#writer_magic.flush()
%tensorboard --logdir 'logs'